[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/chatgpt/plugins/langchain-docs-plugin.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/generation/chatgpt/plugins/langchain-docs-plugin.ipynb)

# Building a LangChain Docs Plugin for ChatGPT

In this walkthrough we setup a ChatGPT plugin.

Before running this notebook you should have already initialized the retrieval API and have it running locally or elsewhere (like on Digital Ocean). More detailed instructions for the setup and deployment can be [found in the video here](https://youtu.be/hpePPqKxNq8).

We will summarize the instructions (specific to the Pinecone datastore) before moving on to the walkthrough.

## App Quickstart

1. Install Python 3.10 if not already installed.

2. Clone the `chatgpt-retrieval-plugin` repository:

```
git clone git@github.com:openai/chatgpt-retrieval-plugin.git
```

_**Note**: To see how we setup the *hosted app* on DigitalOcean [refer to this video](https://youtu.be/hpePPqKxNq8), otherwise continue to setup the app locally by following the remaining steps._

3. Navigate to the app directory:

```
cd /path/to/chatgpt-retrieval-plugin
```

4. Install `poetry`:

```
pip install poetry
```

5. Create a new virtual environment:

```
poetry env use python3.10
```

6. Install the `retrieval-app` dependencies:

```
poetry install
```

7. Set app environment variables:

* `BEARER_TOKEN`: Secret token used by the app to authorize incoming requests. We will later include this in the request `headers`. The token can be generated however you prefer, such as using [jwt.io](https://jwt.io/).

* `OPENAI_API_KEY`: The OpenAI API key used for generating embeddings with the `text-embedding-ada-002` model. [Get an API key here](https://platform.openai.com/account/api-keys)!

8. Set Pinecone-specific environment variables:

* `DATASTORE`: set to `pinecone`.

* `PINECONE_API_KEY`: Set to your Pinecone API key. This requires a free Pinecone account and can be [found in the Pinecone console](https://app.pinecone.io/).

* `PINECONE_ENVIRONMENT`: Set to your Pinecone environment, looks like `us-east1-gcp`, `us-west1-aws`, and can be found next to your API key in the [Pinecone console](https://app.pinecone.io/).

* `PINECONE_INDEX`: Set this to your chosen index name. The name you choose is your choice, we just recommend setting it to something descriptive like `"openai-retrieval-app"`. *Note that index names are restricted to alphanumeric characters, `"-"`, and can contain a maximum of 45 characters.*

8. Run the app with:

```
poetry run start
```

If running the app locally you should see something like:

```
INFO:     Uvicorn running on http://0.0.0.0:8000
INFO:     Application startup complete.
```

In that case, the app has automatically connected to our index (specified by `PINECONE_INDEX`), if no index with that name existed beforehand, the app creates one for us.

Now we're ready to move on to populating our index with some data.

## Required Libraries

There are a few Python libraries we must `pip install` for this notebook to run, those are:

In [1]:
!pip install -qU langchain tiktoken tqdm bs4 boto3


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Preparing Data

In this example, we will download the LangChain docs from [langchain.readthedocs.io/](https://langchain.readthedocs.io/latest/en/). We get all `.html` files located on the site like so:

In [ ]:
!wget -r -A.html -P rtdocs https://langchain.readthedocs.io/en/latest/

This downloads all HTML into the `rtdocs` directory. Now we can use LangChain itself to process these docs. We do this using the `ReadTheDocsLoader` like so:

In [10]:
from langchain.document_loaders import ReadTheDocsLoader

loader = ReadTheDocsLoader('rtdocs',encoding='utf-8')
docs = loader.load()
len(docs)

c:\Users\ydebray\Downloads\chatgpt-retrieval-plugin\env\lib\site-packages\langchain\document_loaders\readthedocs.py:30: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 30 of the file c:\Users\ydebray\Downloads\chatgpt-retrieval-plugin\env\lib\site-packages\langchain\document_loaders\readthedocs.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  _ = BeautifulSoup(
c:\Users\ydebray\Downloads\chatgpt-retrieval-plugin\env\lib\site-packages\langchain\document_loaders\readthedocs.py:46: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). Thi

398

This leaves us with `389` processed doc pages. Let's take a look at the format each one contains:

In [11]:
docs[0]

Document(page_content='', lookup_str='', metadata={'source': 'rtdocs\\langchain.readthedocs.io\\robots.txt.tmp'}, lookup_index=0)

We access the plaintext page content like so:

In [15]:
print(docs[0].page_content)

In [13]:
print(docs[5].page_content)

.md
.pdf
Glossary
 Contents 
Chain of Thought Prompting
Action Plan Generation
ReAct Prompting
Self-ask
Prompt Chaining
Memetic Proxy
Self Consistency
Inception
MemPrompt
Glossary#
This is a collection of terminology commonly used when developing LLM applications.
It contains reference to external papers or sources where the concept was first introduced,
as well as to places in LangChain where the concept is used.
Chain of Thought Prompting#
A prompting technique used to encourage the model to generate a series of intermediate reasoning steps.
A less formal way to induce this behavior is to include “Let’s think step-by-step” in the prompt.
Resources:
Chain-of-Thought Paper
Step-by-Step Paper
Action Plan Generation#
A prompt usage that uses a language model to generate actions to take.
The results of these actions can then be fed back into the language model to generate a subsequent action.
Resources:
WebGPT Paper
SayCan Paper
ReAct Prompting#
A prompting technique that combines Chain-o

We can also find the source of each document:

In [17]:
docs[5].metadata['source'].replace('rtdocs\\', 'https://')

'https://langchain.readthedocs.io\\en\\latest\\glossary.html'

Looks good, we need to also consider the length of each page with respect to the number of tokens that will reasonably fit within the window of a ChatGPT model. We will use `gpt-3.5-turbo` as the assumed model.

### Chunking the Text

At the time of writing, `gpt-3.5-turbo` supports a context window of 4096 tokens — that means that input tokens + generated ( / completion) output tokens, cannot total more than 4096 without hitting an error.

So we 100% need to keep below this. If we assume a very safe margin of ~2000 tokens for the input prompt into `gpt-3.5-turbo`, leaving ~2000 tokens for conversation history and completion.

With this ~2000 token limit we may want to include *five* snippets of relevant information, meaning each snippet can be no more than **400** token long.

To create these snippets we use the `RecursiveCharacterTextSplitter` from LangChain. To measure the length of snippets we also need a *length function*. This is a function that consumes text, counts the number of tokens within the text (after tokenization using the `gpt-3.5-turbo` tokenizer), and returns that number. We define it like so:

In [18]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

Note that for the tokenizer we defined the encoder as `"cl100k_base"`. This is a specific tiktoken encoder which is used by `gpt-3.5-turbo`. Other encoders exist and at the time of writing are summarized as:

| Encoder | Models |
| --- | --- |
| `cl100k_base` | `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002` |
| `p50k_base` | `text-davinci-003`, `code-davinci-002`, `code-cushman-002` |
| `r50k_base` | `text-davinci-001`, `davinci`, `text-similarity-davinci-001` |
| `gpt2` | `gpt2` |

You can find these details in the [Tiktoken `model.py` script](https://github.com/openai/tiktoken/blob/main/tiktoken/model.py), or using `tiktoken.encoding_for_model`:

In [19]:
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

With the length function defined we can initialize our `RecursiveCharacterTextSplitter` object like so:

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

Then we split the text for a document like so:

In [21]:
chunks = text_splitter.split_text(docs[5].page_content)
len(chunks)

2

In [22]:
tiktoken_len(chunks[0]), tiktoken_len(chunks[1])

(346, 247)

For `docs[5]` we created `2` chunks of token length `346` and `247`.

This is for a single document, we need to do this over all of our documents. While we iterate through the docs to create these chunks we will reformat them into the format required by our API app. This format needs to align to the `/upsert` endpoints required document format, which looks like this:

```json
[
    {
        "id": "abc",
        "text": "some important document text",
        "metadata": {
            "field1": "optional metadata goes here",
            "field2": 54
        }
    },
    {
        "id": "123",
        "text": "some other important text",
        "metadata": {
            "field1": "another metadata",
            "field2": 71,
            "field3": "not all metadatas need the same structure"
        }
    }
    ...
]
```

Every document *must* have a `"text"` field. The `"id"` and `"metadata"` fields are optional, however, we will include both.

The `"id"` will be created based on the URL of the text + it's chunk number.

In [65]:
import hashlib
m = hashlib.md5()  # this will convert URL into unique ID

url = docs[5].metadata['source'].replace('rtdocs\\', 'https://').replace('\\','/')
print(url)

# convert URL to unique ID
m.update(url.encode('utf-8'))
uid = m.hexdigest()[:12]
print(uid)

https://langchain.readthedocs.io/en/latest/glossary.html
d5e95d45160a


Then use the `uid` alongside chunk number and actual `url` to create the format needed:

In [41]:
data = [
    {
        'id': f'{uid}-{i}',
        'text': chunk,
        'metadata': {'url': url}
    } for i, chunk in enumerate(chunks)
]
data

[{'id': '337495bbf625-0',
  'text': 'Source code for langchain.vectorstores.weaviate\n"""Wrapper around weaviate vector database."""\nfrom __future__ import annotations\nfrom typing import Any, Dict, Iterable, List, Optional\nfrom uuid import uuid4\nfrom langchain.docstore.document import Document\nfrom langchain.embeddings.base import Embeddings\nfrom langchain.vectorstores.base import VectorStore\n[docs]class Weaviate(VectorStore):\n    """Wrapper around Weaviate vector database.\n    To use, you should have the ``weaviate-client`` python package installed.\n    Example:\n        .. code-block:: python\n            import weaviate\n            from langchain.vectorstores import Weaviate\n            client = weaviate.Client(url=os.environ["WEAVIATE_URL"], ...)\n            weaviate = Weaviate(client, index_name, text_key)\n    """\n    def __init__(\n        self,\n        client: Any,\n        index_name: str,\n        text_key: str,\n        attributes: Optional[List[str]] = None,\

Now we repeat the same logic across our full dataset:

In [52]:
from tqdm.auto import tqdm

documents = []

for doc in tqdm(docs):
    url = doc.metadata['source'].replace('rtdocs\\', 'https://').replace('\\','/')
    m.update(url.encode('utf-8'))
    uid = m.hexdigest()[:12]
    chunks = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks):
        documents.append({
            'id': f'{uid}-{i}',
            'text': chunk,
            'metadata': {'url': url}
        })

len(documents)

100%|██████████| 398/398 [00:05<00:00, 78.87it/s] 


2237

In [53]:
documents[0]

{'id': '8ab454e6df9f-0',
 'text': '.md\n.pdf\nDeployments\n Contents \nStreamlit\nGradio (on Hugging Face)\nBeam\nVercel\nSteamShip\nDeployments#\nSo you’ve made a really cool chain - now what? How do you deploy it and make it easily sharable with the world?\nThis section covers several options for that.\nNote that these are meant as quick deployment options for prototypes and demos, and not for production systems.\nIf you are looking for help with deployment of a production system, please contact us directly.\nWhat follows is a list of template GitHub repositories aimed that are intended to be\nvery easy to fork and modify to use your chain.\nThis is far from an exhaustive list of options, and we are EXTREMELY open to contributions here.\nStreamlit#\nThis repo serves as a template for how to deploy a LangChain with Streamlit.\nIt implements a chatbot interface.\nIt also contains instructions for how to deploy this app on the Streamlit platform.\nGradio (on Hugging Face)#\nThis repo se

We're now left with `2237` documents in the format required by our API.

---

#### (Optional) Load Dataset from Hugging Face

Rather than running the above scripts to build the dataset, you can load a prepared version from Hugging Face Datasets like so:

In [29]:
!pip install -qU datasets

from datasets import load_dataset

documents = load_dataset('jamescalam/langchain-docs', split='train')
documents


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 90.90it/s]
                                                        

Dataset json downloaded and prepared to C:/Users/ydebray/.cache/huggingface/datasets/jamescalam___json/jamescalam--langchain-docs-bcc23a7c6d742f0e/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'text', 'source'],
    num_rows: 2212
})

In [30]:
documents[0]

{'id': '9c04de564ed3-0',
 'text': '.rst\n.pdf\nWelcome to LangChain\n Contents \nGetting Started\nModules\nUse Cases\nReference Docs\nLangChain Ecosystem\nAdditional Resources\nWelcome to LangChain#\nLarge language models (LLMs) are emerging as a transformative technology, enabling\ndevelopers to build applications that they previously could not.\nBut using these LLMs in isolation is often not enough to\ncreate a truly powerful app - the real power comes when you are able to\ncombine them with other sources of computation or knowledge.\nThis library is aimed at assisting in the development of those types of applications. Common examples of these types of applications include:\n❓ Question Answering over specific documents\nDocumentation\nEnd-to-end Example: Question Answering over Notion Database\n💬 Chatbots\nDocumentation\nEnd-to-end Example: Chat-LangChain\n🤖 Agents\nDocumentation\nEnd-to-end Example: GPT+WolframAlpha\nGetting Started#\nCheckout the below guide for a walkthrough of ho

This needs to be reformated into the format we need for the API:

In [31]:
documents = [{
    'id': doc['id'],
    'text': doc['text'],
    'metadata': {'url': doc['source']}
} for doc in documents]

documents[0]

{'id': '9c04de564ed3-0',
 'text': '.rst\n.pdf\nWelcome to LangChain\n Contents \nGetting Started\nModules\nUse Cases\nReference Docs\nLangChain Ecosystem\nAdditional Resources\nWelcome to LangChain#\nLarge language models (LLMs) are emerging as a transformative technology, enabling\ndevelopers to build applications that they previously could not.\nBut using these LLMs in isolation is often not enough to\ncreate a truly powerful app - the real power comes when you are able to\ncombine them with other sources of computation or knowledge.\nThis library is aimed at assisting in the development of those types of applications. Common examples of these types of applications include:\n❓ Question Answering over specific documents\nDocumentation\nEnd-to-end Example: Question Answering over Notion Database\n💬 Chatbots\nDocumentation\nEnd-to-end Example: Chat-LangChain\n🤖 Agents\nDocumentation\nEnd-to-end Example: GPT+WolframAlpha\nGetting Started#\nCheckout the below guide for a walkthrough of ho

---

### Indexing the Docs

We're now ready to begin indexing (or *upserting*) our `documents`. To make these requests to the retrieval app API, we will need to provide authorization in the form of the `BEARER_TOKEN` we set earlier. We do this below:

In [43]:
import os
from dotenv import load_dotenv

load_dotenv()
BEARER_TOKEN = os.environ.get("BEARER_TOKEN") 

Use the `BEARER_TOKEN` to create our authorization `headers`:

In [44]:
headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}"
}

We'll perform the upsert in batches of `batch_size`. Make sure that the `endpoint_url` variable is set to the correct location for your running *retrieval-app* API.

In [55]:
import requests
from requests.adapters import HTTPAdapter, Retry
from tqdm.auto import tqdm

batch_size = 100
endpoint_url = "https://seal-app-ulquy.ondigitalocean.app"
s = requests.Session()

# we setup a retry strategy to retry on 5xx errors
retries = Retry(
    total=5,  # number of retries before raising error
    backoff_factor=0.1,
    status_forcelist=[500, 502, 503, 504]
)
s.mount('http://', HTTPAdapter(max_retries=retries))

for i in tqdm(range(0, len(documents), batch_size)):
    i_end = min(len(documents), i+batch_size)
    # make post request that allows up to 5 retries
    res = s.post(
        f"{endpoint_url}/upsert",
        headers=headers,
        json={
            "documents": documents[i:i_end]
        }
    )

100%|██████████| 23/23 [04:19<00:00, 11.29s/it]


Manage Pinecone database
- https://docs.pinecone.io/docs/python-client
- https://docs.pinecone.io/docs/manage-data
- https://docs.pinecone.io/reference/delete_post

In [56]:
import pinecone 

pinecone.init(api_key=os.environ['PINECONE_API_KEY'], environment=os.environ['PINECONE_API_ENV']) 
# pinecone.list_indexes()
index = pinecone.Index('chatgpt-plugin') 

fetch_response = index.fetch(ids=['8ab454e6df9f-0', '8ab454e6df9f-1'])
fetch_response

# delete_response = index.delete(deleteAll=True)
# delete_response

{'namespace': '', 'vectors': {}}

With that our LangChain doc records have all been indexed and we can move on to querying.

### Making Queries

To query the datastore all we need to do is pass one or more queries to the `/query` endpoint. We can make a few questions related to LangChain and see if we return relevant info:

In [57]:
queries = [
    {'query': "What is the LLMChain in LangChain?"},
    {'query': "How do I use Pinecone in LangChain?"},
    {'query': "What is the difference between Knowledge Graph memory and buffer memory for "+
     "conversational memory?"}
]

res = requests.post(
    f"{endpoint_url}/query",
    headers=headers,
    json={
        'queries': queries
    }
)
res

<Response [200]>

In [63]:
query = [{'query': "What is the LLMChain in LangChain?"}]

res = requests.post(
    f"{endpoint_url}/query",
    headers=headers,
    json={
        'queries': query
    }
)
res

<Response [200]>

Now we can loop through the responses and see the results returned for each query:

In [64]:
for query_result in res.json()['results']:
    query = query_result['query']
    answers = []
    scores = []
    for result in query_result['results']:
        answers.append(result['text'])
        scores.append(round(result['score'], 2))
    print("-"*70+"\n"+query+"\n\n"+"\n".join([f"{s}: {a}" for a, s in zip(answers, scores)])+"\n"+"-"*70+"\n\n")

----------------------------------------------------------------------
What is the LLMChain in LangChain?

0.91: Links Used: LLMChain
0.88: nThese are, in increasing order of complexity:\n\nð\x9f“\x83 LLMs and Prompts:\n\nThis includes prompt management, prompt optimization, generic interface for all LLMs, and common utilities for working with LLMs.\n\nð\x9f”\x97 Chains:\n\nChains go beyond just a single LLM call, and are sequences of calls (whether to an LLM or a different utility). LangChain provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications.
0.88: Chains: Chains go beyond just a single LLM call, and are sequences of calls (whether to an LLM or a different utility). LangChain provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications. Indexes: Language models are often more powerful when combined with your own text data - this module covers b

The top results are all relevant as we would have hoped. With that we've finished. The retrieval app API can be shut down, and to save resources the Pinecone index can be deleted within the [Pinecone console](https://app.pinecone.io/).